KoBART를 활용해서 코인뉴스 한줄 요약하는 예시를 보여준다

In [1]:
import numpy as np 
import pandas as pd
import torch 
from transformers import * 

PyTorch version 1.7.1 available.
TensorFlow version 2.5.0 available.


In [2]:
### define KoBART model ### 
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   

In [3]:
### text scraped from coinness ### 
text = '''
솔라나 지갑 팬텀, 모바일 앱 내일 출시
코인데스크에 따르면, 솔라나 지갑 팬텀(Phantom)이 30일 iOS 베타 버전을 출시하고, 한달 안에 전체 버전을 출시할 예정이다. 모바일 지갑은 암호화폐와 NFT 거래, 전송, 스왑, 스테이킹 등을 지원한다. 팬텀 지갑은 11월 100만 명의 활성 사용자를 유치, 모바일 버전 없이 솔플레어(SolFlare) 지갑 사용자 수를 제친 바 있다.
'''

### tokenize text for preprocessing ### 
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id] 

### generate summary using KoBART ### 
summary_ids = model.generate(torch.tensor([input_ids]),
                             max_length=256,
                             early_stopping=True,
                             repetition_penalty=2.0)

summ = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True) 

print("원문:")
print(text)

print()

print("요약문:")
print(summ)

원문:

솔라나 지갑 팬텀, 모바일 앱 내일 출시
코인데스크에 따르면, 솔라나 지갑 팬텀(Phantom)이 30일 iOS 베타 버전을 출시하고, 한달 안에 전체 버전을 출시할 예정이다. 모바일 지갑은 암호화폐와 NFT 거래, 전송, 스왑, 스테이킹 등을 지원한다. 팬텀 지갑은 11월 100만 명의 활성 사용자를 유치, 모바일 버전 없이 솔플레어(SolFlare) 지갑 사용자 수를 제친 바 있다.


요약문:
30일 iOS 베타 버전을 출시하는 솔라나 지갑 팬텀(Phantom)은 암호화폐와 NFT 거래, 전송 등을 지원하는 모바일 앱 전체버전도 선보일 예정이다.
